# ALMA Science Archive School 2022 (Exercises)
#### Authors: Toma Badescu, George Bendo, Aida Ahmadi, Ashley Bemis

 * This hands-on session is part of the first ALMA Science Archive School taking place in Bologna, Italy, on Oct. 5-7, 2022. 

 * It contains exercises that cover some of the ways one can query the [ALMA Science Archive (ASA)](https://almascience.eso.org/aq/). 

 * Each cell can be executed by pressing <kbd>Shift</kbd>+<kbd>Enter</kbd>. 
 
 * You may enter your attempts in the empty cells and create new cells by clicking the <kbd>+</kbd> button.
 
This exercise sheet is divided into four parts as follows. In the first three you get to work on some exercises that highlight some of the ways one can query and mine the ALMA Science Archive. In the last part, you get to work on your own research problem and we are happy to help you with your specific case. Feel free to jump between topics and use the allotted time as it is most efficient for you.

### [Part 1. pyVO](#pyVO)
### [Part 2. astroquery](#astroquery)
### [Part 3. ALminer](#alminer)
### [Part 4. Your own workstation](#workstation)

# Part 1. pyVO<a name="pyVO"></a>

pyVO is a python Virtual Observatory (VO) package that is designed for searching and accessing data from archives that use VO standards. Although astroquery can also be used within python to query the ALMA archive, pvVO is the only python tool that is officially supported by the ALMA archive developers.

The package can be installed using `pip install pyvo`.

This section covers the following topics:

* Searching by various parameters (including target position, frequency, and polarization).
* Using wildcards (%).
* Resolving coordinate names and searching at those coordinates.
* Searching using multiple criteria.
* Returning only a subset of available columns in the search results.


### Resources:

* For help on the language that is used in setting up queries see [the ADQL documentation page](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html).
* For examples on using pyVO to query the ALMA archive see [the series of notebooks on the ALMA Science Portal](https://almascience.eso.org/alma-data/archive/archive-notebooks).


### PyVO set-up

Aside from the standard import command, an additional command is needed to set up access to the archvie.  Once this is set up, it is possible to perform a large series of searches.

In [ ]:
# Import pvVO:
import pyvo

# Set up access to the ESO archive:
service = pyvo.dal.TAPService("https://almascience.eso.org/tap")

# If, for whatever reason, the ESO archive is down, you may use the NRAO/NAOJ mirrors:
# service = pyvo.dal.TAPService("https://almascience.nrao.edu/tap")
# service = pyvo.dal.TAPService("https://almascience.nao.ac.jp/tap")

### Example 1.1: Search for the coordinates RA=204.253958 and Dec=-29.865417, with a radius of 0.006 degrees

Note that the output from the query should be converted to a [PANDAS DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html). The pandas.DataFrame object has multiple columns with quantities that can be either printed or used for multiple other purposes. A complete list of the columns in the object can be displayed using 'pandas.DataFrame.columns'. These columns can also be used to perform other queries as you will see further below.

#### Display the output query table:

Note that if you want to display all rows/columns you need to run the following cells first (but beware this may overload your browser and crash the notebook!). Instead, you can save tables to files that can be opened by other programs and read in later too. In the case of pandas.DataFrame, this can be done with [pandas.DataFrame.to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) function.

In [ ]:
# import pandas as pd
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

#### Display all the column names in the output query:

#### Display the unique proposal IDs associated with the search results:

#### Plot the right ascension and declination of all fields in the search results using `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt

### Example 1.2: Search by ALMA project code (e.g. ALMA Large Program PHANGS 2017.1.00886.L)

### Example 1.3: Search for full polarization data & print unique project codes

### Example 1.4: Search for all programs from Cycle 6 proposals (project codes beginning with 2018) & print unique project codes

### Example 1.5: Search for data by frequency (e.g. all observations covering the range 115.20 to 115.35 GHz) & print unique project codes

### Example 1.6: Resolve a source name and search at that position
Note that this is not the same as searching by the target name used in the ALMA observations. 

#### First use `astropy` to obtain the position of the target "Cen A":

In [ ]:
import astropy
coordinates = astropy.coordinates.SkyCoord.from_name("Cen A")
ra=float(coordinates.ra.degree)
dec=float(coordinates.dec.degree)

#### Now query for the obtained position within a search radius of 0.006 degrees and print the unique project codes:

### Example 1.7: Query all Band 3 full polarization data from Cycle 7

### Example 1.8: Query by both position and frequency constraints (e.g. combining examples 1.1 and 1.5) but only return the following four columns in the output: proposal_id, member_ous_uid, frequency, bandwidth

### Example 1.9: Download data associated with a specific MOUS (e.g. uid___A001_X135b_X6b)

Downloading data with pyVO is relatively straightforward, although it is primarily done by one Member OUS at a time.  

Note that the Member OUSs need to be formatted with underscores (\_), whereas the member_ous_id column from pyVO searches are in a different format (see the table in the previous example).  The colon (:) and slashes (/) in the pyVO search results need to be replaced with underscores in this command, as shown in this example:

pyVO search output: uid://A001/X135b/X6b

pyVO download string: uid\_\_\_A001\_X135b\_X6b

You should now be able to see some tar files downloaded in your current working directory.

# Part 2. Astroquery<a name="astroquery"></a>

Astroquery is a set of tools for querying astronomical web forms and databases. 

It can be installed using `pip install --pre astroquery`

In this part we use astroquery's catalog and archive search functionality in addition to its line list services to find recent ALMA data on the CO 1-0 transition across M83. This can be adapted to similar extragalactic projects.

This section covers the following exercises:
1. [Perform an object query of the ALMA science archive](#object_query)
2. [Use source redshift to check which ALMA band covers line](#using_redshift)
3. [Download ALMA data using Astroquery](#download_data)
4. [Send TAP queries (similar to pyVO)](#astroquery_tap)
5. [Compare catalogs](#compare_catalogs)

### Resources:
- If you want to access help for a particular command, type `help(command)` and run the cell
- The Astroquery readthedocs is located [here](https://astroquery.readthedocs.io/en/latest/).
- We suggest running Astroquery version 0.4.7. Some steps of this tutorial will not work in earlier versions, such as HSA search functionionality or ALMA authentication.
- The output of astroquery searches are astropy tables (see [astropy tables documentation page](https://docs.astropy.org/en/stable/table/index.html) for more information on how to work with them).

### Astroquery set-up

In [ ]:
# Import Astroquery and check your current working version by doing the following:
import astroquery
print("astroquery version: {}".format(astroquery.__version__))

# Import astroquery's ALMA archive search tool and other useful packages:
from astroquery.alma import Alma
import numpy as np
import astropy.units as u

#instantiate alma object/class
alma = Alma()

# If you have access to proprietary data that you want to work with, you will need to login to your ALMA account:
#login_name = ""
#alma.login(login_name)

<a id="object_query"></a>
### Example 2.1 Perform an object query of the ALMA science archive

We perform an ALMA object query (`alma.query_object`). Astroquery sends the source name to the ALMA Archive, which then resolves the source coordinates using, i.e. Simbad, NED, and Vizier, for matches to the object name. Astroquery returns an astropy table containing information on all of the ALMA archival data towards M83's coordinates.

Note that you can instead perform a region search (`alma.query_region`) using Astroquery specifying a search radius. The default ALMA search radius is 10 arcminutes.


#### Use an object query to search the ALMA archive for M83 observations & print the resulting astropy table:

#### Print the column names of the resulting table:

#### Select observations by time and ALMA band (e.g. bands 7 & 9 and data that is newer than 2017):

In [ ]:
from datetime import datetime

<a id="using_redshift"></a>
### Example 2.2 Use source redshift to check which ALMA band covers line

Although M83 is fairly close ($z<<1$), it is useful to calculate the redshifted frequency of transitions we are intereted in to be able to carefully check if they fall within the spectral setup of the ALMA observations in the archive.

#### First query NED to grab the photometric redshift of M83:

In [ ]:
from astroquery.ipac.ned import Ned

#### Use Splatalogue to get the rest frequencies of our transition(s) of interest, i.e. CO v=0 lines:

In [ ]:
from astroquery.splatalogue import Splatalogue

#### Use ALMA utils to compare the frequency support of the archival ALMA data on M83 with the redshifted frequency of CO v=0 lines:

In [ ]:
from astroquery.alma.utils import parse_frequency_support

<a id="download_data"></a>
### Example 2.3 Download ALMA data associated with the previous example using Astroquery

Once you have decided which data you would like to work with locally, you can download it from the archive using the `access_url` provided in the astropy table. `alma.get_data_info` can be used to further list what is in each tarfile in the archive.

#### First find only results with unique `member_ous_uid` in order to avoid downloading duplicates:

In [ ]:
from astropy.table import unique

#### List what is in the first two archives (to cut down on time):
(Note: this may take some time)

#### Make a list of only the FITS files and print the URLs:

#### Set a local cache for the data and download:

<a id="astroquery_tap"></a>
### Example 2.4: Use astroquery to send TAP queries (similar to pyVO)

### Query all observations with central frequencies between 300 and 400 GHz, sensitivity better than 0.5 mJy/beam @ 10 km/s, and science_keyword containing the word 'disk'

<a id="compare_catalogs"></a>
### Example 2.5: Compare data catalogs

#### First perform a conesearch query (~1 degree) at the position of the COSMOS field centre (RA=10h00m24s, Dec=+02d10m55s) to the ALMA Science Archive:

#### Use astroquery to query the Herschel Science Archive:

#### Find what column names are associated with the RA/Dec in the Herschel results:

#### Cross-match the two catalogs:

# Part 3: ALminer<a name="alminer"></a>

ALminer is a Python-based code to effectively query, analyse, and visualize the ALMA Science Archive (ASA). It uses pyVO to query the ASA in a simple and easy-to-use way and has useful visualisations that help with filtering the queries further. It also allows users to directly download ALMA data products and/or raw data for further image processing.

The package can be installed using `pip install alminer`.

### Resources:

- [ALminer documentation](https://alminer.readthedocs.io/en/latest/) can be found on readthedocs.
- The most recent version of the code can be found on [GitHub](https://github.com/emerge-erc/ALminer).
- It contains an [extensive tutorial Jupyter Notebook](https://nbviewer.org/github/emerge-erc/ALminer/blob/main/notebooks/tutorial/ALminer_tutorial.ipynb?flush_cache=True) where all alminer features have been highlighted.
- Here is a list of all other possible [scientific categories](https://alminer.readthedocs.io/en/latest/pages/scientific_categories.html) and [keywords](https://alminer.readthedocs.io/en/latest/pages/science_keywords.html) for the `alminer.keysearch` function. 

### ALminer setup

In [ ]:
import alminer

### Example 3.1: Query a list of protoplanetary disks by their names (e.g. HL Tau, AB Aur, DG Tau)
* Note that the names must be known objects in SIMBAD, VizieR, or NED
* Use the NRAO service for the moment.

#### Display the query results as a table, and show all rows and columns:

### Example 3.2: Query an object by its coordinates (RA=201.365063, Dec=-43.019112) and a search radius of 10 arcmin

### Example 3.3: Query for full polarization data in the science keyword 'disks around low-mass stars'

### Example 3.4: Search for all projects in the scientific category 'Galaxy evolution' that contain the words 'star formation' AND outflow in their abstracts

Note that this is a large query that may take a while to run. Make sure to turn of the feature that prints the target names.

### Example 3.5: Search for all projects in the scientific category 'Galaxy evolution' that contain the words 'star formation' OR outflow in their abstracts

#### Plot an overview of the query results:

### Example 3.6: Filter the results of the query in the previous example to contain only observations with angular resolutions < 3" & velocity resolution < 60 km/s
Hint: The query results is a pandas.DataFrame. You can print all columns by `pandas.DataFrame.columns` command where pandas.DataFrame is your query results. To get the description and units of a given column use the `alminer` command `alminer.get_info(column_name)`.

#### Print the summary and plot the overview again, highlighting CO & its isotopologues:

### Example 3.7: Search whether any CO lines and its isotopologues are covered in the above observations
Note that if you are interested in observations of targets at high-redshift, you can specify a redshift using the 'z' parameter.

### Example 3.8: Search whether a given frequency is covered in the observations

#### Highlight this frequency in the overview plots:

### Example 3.9: Download all data products (raw + products) pertaining to the observations in example 3.7

### Example 3.10: Download only continuum FITS images for the science target pertaining to the observations in example 3.8

### Example 3.11: Send a pyVO query using ALminer (e.g. the same query as Example 2.4) and convert it to ALminer format

Now you can use all the analysis and plotting routines presented before on these observations.

# Part 4: Your workstation

In this section you can make your own queries by using any of the introduced methods. Feel free to request help from any of us in the room. 

Happy mining!